In [ ]:
!pip install awscli
!pip install boto3

In [ ]:
import os
import sys
from google.colab import userdata

os.environ['GITTOKEN'] = userdata.get('github_astro_guard_token')

!rm -r sample_data
!git clone https://${GITTOKEN}@github.com/zLPrime/astro-guard.git

sys.path.append('./astro-guard/ml-workbench')

rm: cannot remove 'sample_data': No such file or directory
fatal: destination path 'astro-guard' already exists and is not an empty directory.


In [ ]:
from astro_data import dataloader
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

train_data, test_data = dataloader.get_train_test(dataloader.get_jd_magn_graph_dataset(
    userdata.get('aws_access_key_id'),
    userdata.get('aws_secret_access_key'), label_encoder,
    cache_path='/content/astro-guard/ml-workbench/astro_data/objects_cache.json'),
                                                  batch_size=16)

Loading object keys from cache...


In [ ]:
from astro_model.model import SimpleCNN
from astro_train.trainer import astro_train
from torch import cuda

model = SimpleCNN()
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import resource

# Set max memory usage to 1 GB (in bytes)
ONE_GB = 1 * 1024 * 1024 * 1024
resource.setrlimit(resource.RLIMIT_AS, (ONE_GB * 30, ONE_GB * 30))

In [ ]:
astro_train(model, train_data, 3, device)

 67%|██████▋   | 59/88 [18:59<09:27, 19.58s/it]

In [ ]:
from astro_test.test import print_evaluate

print_evaluate(model, test_data, label_encoder, device)